In [1]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in c:\users\lenovo\anaconda3\lib\site-packages (from requests<3.0.0dev,>=2.18.0->google-api-core<2dev,>=1.21.0->google-api-python-client) (1.25.11)



In [2]:
pip install google-auth google-auth-oauthlib google-auth-httplib2

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install demoji

In [4]:
!pip install langdetect

  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993197 sha256=d5112ad58f71245f9fb9996ce135f4acc55d4c9df1552a6e8e842a1c3242976c
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\1e\80\23\0a24928ec3a3906ff5027f38d2fea824e7e97f2ba7c83d91e3
Successfully built langdetect


In [38]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import demoji
from langdetect import detect
import re   # regular expression

In [39]:
CLIENT_SECRETS_FILE = "client_secret.json"

In [40]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [41]:
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [56]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

<IPython.core.display.Javascript object>

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=467921254610-jdtkvo5en84fuvemk89cgb69n5oam8bm.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=3lN7GRs6QTSH0diGKbFaUtZBDsnQaF&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g5rFhAfo7Akg9MTYi1jBK0hXhe-mkDMuVxEc7zO2uwFKfVuvm7U8a0


In [57]:
query = "CM YS Jagan Orders And Precautionary Measures To Control Coronavirus Spread | 104 Services"

In [58]:
query_results = service.search().list(part = 'snippet',q = query,
                                      order = 'relevance', 
                                      type = 'video',
                                      relevanceLanguage = 'en',
                                      safeSearch = 'moderate').execute()

In [46]:
query_results['items']

[{'kind': 'youtube#searchResult',
  'etag': 'Zq9-HEp0LHlVZ-Ind7wdd34c7IY',
  'id': {'kind': 'youtube#video', 'videoId': 'WB-y7_ymPJ4'},
  'snippet': {'publishedAt': '2021-03-12T15:00:06Z',
   'channelId': 'UCZFMm1mMw0F81Z37aaEzTUA',
   'title': 'NDTV 24x7 LIVE TV - Watch Latest News in English | Breaking News',
   'description': 'NDTV 24x7 live stream is also available on https://www.ndtv.com/live. About Us: NDTV brings you unbiased and comprehensive coverage of news and ...',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/WB-y7_ymPJ4/default_live.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/WB-y7_ymPJ4/mqdefault_live.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/WB-y7_ymPJ4/hqdefault_live.jpg',
     'width': 480,
     'height': 360}},
   'channelTitle': 'NDTV',
   'liveBroadcastContent': 'live',
   'publishTime': '2021-03-12T15:00:06Z'}},
 {'kind': 'youtube#searchResult',
  'etag': 

In [59]:
video_id = []
channel = []
video_title = []
video_desc = []
for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])

In [60]:
video_id = video_id[0]
channel = channel[0]
video_title = video_title[0]
video_desc = video_desc[0]

In [61]:
video_id[0]

'q'

In [62]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []


comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []

In [63]:
nextPage_token = None

while 1:
    response = service.commentThreads().list(
                    part = 'snippet',
                    videoId = video_id,
                    maxResults = 100, 
                    order = 'relevance', 
                    textFormat = 'plainText',
                    pageToken = nextPage_token
                    ).execute()


    nextPage_token = response.get('nextPageToken')
    for item in response['items']:
        comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
        comment_id_temp.append(item['snippet']['topLevelComment']['id'])
        reply_count_temp.append(item['snippet']['totalReplyCount'])
        like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
        comments_pop.extend(comments_temp)
        comment_id_pop.extend(comment_id_temp)
        reply_count_pop.extend(reply_count_temp)
        like_count_pop.extend(like_count_temp)
        video_id_pop.extend([video_id]*len(comments_temp))
        channel_pop.extend([channel]*len(comments_temp))
        video_title_pop.extend([video_title]*len(comments_temp))
        video_desc_pop.extend([video_desc]*len(comments_temp))

    if nextPage_token is  None:
        break

In [64]:
output_dict = {
        'Channel': channel_pop,
        'Video Title': video_title_pop,
        'Video Description': video_desc_pop,
        'Video ID': video_id_pop,
        'Comment': comments_pop,
        'Comment ID': comment_id_pop,
        'Replies': reply_count_pop,
        'Likes': like_count_pop,
        }

output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

In [65]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Channel            28 non-null     object
 1   Video Title        28 non-null     object
 2   Video Description  28 non-null     object
 3   Video ID           28 non-null     object
 4   Comment            28 non-null     object
 5   Comment ID         28 non-null     object
 6   Replies            28 non-null     int64 
 7   Likes              28 non-null     int64 
dtypes: int64(2), object(6)
memory usage: 1.9+ KB


In [66]:
duplicates = output_df[output_df.duplicated("Comment ID")]
print("Count of duplicat comments in dataframe", duplicates.shape[0])

Count of duplicat comments in dataframe 21


In [67]:
print("count of unique comments in dataframe ",output_df.shape[0]- duplicates.shape[0])

count of unique comments in dataframe  7


In [68]:
unique_df = output_df.drop_duplicates(subset=['Comment'])

In [69]:
unique_df.head()

Channel                                        Video Title  \
0   Sakshi TV  CM YS Jagan Orders And Precautionary Measures ...   
2   Sakshi TV  CM YS Jagan Orders And Precautionary Measures ...   
5   Sakshi TV  CM YS Jagan Orders And Precautionary Measures ...   
9   Sakshi TV  CM YS Jagan Orders And Precautionary Measures ...   
14  Sakshi TV  CM YS Jagan Orders And Precautionary Measures ...   

                                    Video Description     Video ID  \
0   CM YS Jagan Orders And Precautionary Measures ...  qGp_ak_SS48   
2   CM YS Jagan Orders And Precautionary Measures ...  qGp_ak_SS48   
5   CM YS Jagan Orders And Precautionary Measures ...  qGp_ak_SS48   
9   CM YS Jagan Orders And Precautionary Measures ...  qGp_ak_SS48   
14  CM YS Jagan Orders And Precautionary Measures ...  qGp_ak_SS48   

                                              Comment  \
0                Inter exams sangathit cheppandi anna   
2   Thanks sir meru Baga work cheystunaru kani adi...   
5                               104 పని చేయడం లేదు...   
9   Jagan Anna please give a message to people. Yo...   
14                               Cancel inter exam ms   

                    Comment ID  Replies  Likes  
0   UgxV6hWMUW2lQ8DO7Zx4AaABAg        0      0  
2   UgzAZTIPENwXqD5EgNR4AaABAg        0      1  
5   UgwbsvHB9DjWx1OBHnt4AaABAg        0      1  
9   UgzSNsq3FS0_Tqv5l_R4AaABAg        0      0  
14  Ugzyq_6LWrJXwOJ27NB4AaABAg        0      2

In [70]:
unique_df.to_csv("extraced_comments.csv",index = False)

In [73]:
pip install pytchat

Note: you may need to restart the kernel to use updated packages.


In [13]:
stream = pytchat.create(video_id = "6E7_A_eY8cg")
i = 0
while stream.is_alive():
    data = stream.get()
    items = data.items
    for c in items:
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
    time.sleep(6)
    i += 1
    if i == 6:
    # get the continuation parameter
        continuation = stream.continuation
        stream.terminate()
        break

# retrieve chatdata from the continuation.
stream = pytchat.create(video_id = "6E7_A_eY8cg", replay_continuation=continuation)
data = stream.get()
items = data.items
for c in items:
    print(f"{c.datetime} [{c.author.name}]- {c.message}")
stream.terminate()


2021-04-28 11:37:52 [The Vision]- kashmiri behn bhaio sy maafi maango unko azad kr do sab sai ho jay ga
2021-04-28 11:38:05 [Adarsh kumar]- sword fish dudh mangoga to khir. dega kashmir mangoga sala sidhe jar se ukhar dega
2021-04-28 11:38:07 [tamoor ramzan]- hahaha kashmir ko dbrdsti india bnana h na mumkin
2021-04-28 11:38:10 [RB]- 🐖💨💨💨💨💨🇵🇰🇵🇰
2021-04-28 11:38:13 [Sanjay Kumar]- :thanksdoc::socialdist:
2021-04-28 11:38:14 [Anita Rajput]- ye tu feko hhhh
2021-04-28 11:38:15 [Nia]- #Noreen I figured that. tu bas raan ban k reh Nabi bhn k loveDe ki ! level up then talk to me 😂 fuddu Jesi bato me slow mod me reply waste nahi KARNA 😂
2021-04-28 11:38:19 [Noreen sabi]- Malika ...Nia Dil par layrahi hah 😂😂😂😋


In [18]:
chat = pytchat.create(video_id="6E7_A_eY8cg")

while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} {c.author.name} {c.message}")
    time.sleep(1)

2021-04-28 11:43:10 vivek surtange reli me mask nhi pahnte or hamre me mast 😂😂😂
2021-04-28 11:43:16 Nagabhushan Bhat test
2021-04-28 11:43:22 Vipul Pandey hindustan zindabaad
2021-04-28 11:43:39 Faith on Allah such hai🇩🇿🇩🇿🇩🇿
2021-04-28 11:43:42 Khushbu Kumari Hindustan jindabad tha jindabad Hai jindabad rahega
2021-04-28 11:43:53 Adarsh kumar yes hindustani zindabad


OverflowError: sleep length is too large